<a href="https://colab.research.google.com/github/hanyeob/fine_dust/blob/merged-scaling/make_scaled_csv_file(%ED%86%B5%ED%95%A9%EC%8A%A4%EC%BC%80%EC%9D%BC%EB%A7%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 불러오기

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
classlist = ['isc_2_1','isc_3_1','isc_3_3', # 0, 1, 2
             'nsc_1_1','nsc_1_2','nsc_2_5', # 3, 4, 5 
             'sdc_1_1','sdc_2_1',           # 6, 7
             'sic_2_6','sic_4_2']           # 8, 9
# ----------------------- 데이터 선택------------------------------
NUMBER = 9;
# ----------------------- 데이터 선택------------------------------

choosefile = classlist[NUMBER]
data_address = 'https://raw.githubusercontent.com/hanyeob/fine_dust/master/csv%20files/' + choosefile +".csv"

In [ ]:
dataset = pd.read_csv(data_address)
dataset[0:5]

,tt,0.3_outdoor,0.5_outdoor,1.0_outdoor,3.0_outdoor,5.0_outdoor,10.0_outdoor,0.3_classroom,0.5_classroom,1.0_classroom,3.0_classroom,5.0_classroom,10.0_classroom,temp_outdoor,RH_outdoor,AH_outside,CO2_outdoor,temp_classroom,RH_classroom,AH_classroom,CO2_classroom,ACH,temp_meteo_425,RH_meteo_425,WD_meteo_425,WS_meteo_425,temp_meteo_889,RH_meteo_889,WD_meteo_889,WS_meteo_889,PM10_air_outdoor,PM2.5_air_outdoor,temp_air_outdoor,RH_air_outdoor,AH_air_outdoor,CO2_air_outdoor,VOCS_air_outdoor,Noise_air_outdoor,PM10_air_classroom,PM2.5_air_classroom,temp_air_classroom,RH_air_classroom,AH_air_classroom,CO2_air_classroom,VOCS_air_classroom,Noise_air_classroom,ΔT
0,9-12-2019 17:21:58,370382300,67033920,3442050,163958,70318,7420,214670300,36660780,2059364,109541,63251,14841,14.0,36,NaN,673,22.0,21,NaN,572,NaN,6.2,62.3,137.4,1.2,7.1,56.7,93.2,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9-12-2019 17:22:58,374975600,68299650,3689046,176325,82686,4594,213868600,36736400,2059717,124735,51590,10247,13.7,35,NaN,667,22.0,21,NaN,571,NaN,6.4,62.3,170.4,2.8,7.2,56.5,66.2,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9-12-2019 17:23:58,371719500,67523320,3385866,152297,66431,8481,212717300,36209540,2102827,112014,54417,9894,13.3,35,NaN,649,22.0,21,NaN,570,NaN,6.4,62.2,163.2,1.1,7.2,56.1,77.5,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9-12-2019 17:24:58,370673200,67153360,3345936,145230,75265,9894,213726200,35985870,2047350,113781,57597,6360,12.5,35,NaN,572,21.9,21,NaN,568,NaN,6.4,62.2,160.0,1.3,7.1,56.3,73.4,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9-12-2019 17:25:58,366765000,66251590,3392933,190813,72792,9187,208327600,35034630,2043463,133922,71025,15194,11.7,35,NaN,538,21.9,21,NaN,567,NaN,6.5,61.9,158.3,2.0,7.1,56.7,82.6,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dataset.columns

Index(['tt', '0.3_outdoor', '0.5_outdoor', '1.0_outdoor', '3.0_outdoor',
       '5.0_outdoor', '10.0_outdoor', '0.3_classroom', '0.5_classroom',
       '1.0_classroom', '3.0_classroom', '5.0_classroom', '10.0_classroom',
       'temp_outdoor', 'RH_outdoor', 'AH_outside', 'CO2_outdoor',
       'temp_classroom', 'RH_classroom', 'AH_classroom', 'CO2_classroom',
       'ACH', 'temp_meteo_425', 'RH_meteo_425', 'WD_meteo_425', 'WS_meteo_425',
       'temp_meteo_889', 'RH_meteo_889', 'WD_meteo_889', 'WS_meteo_889',
       'PM10_air_outdoor', 'PM2.5_air_outdoor', 'temp_air_outdoor',
       'RH_air_outdoor', 'AH_air_outdoor', 'CO2_air_outdoor',
       'VOCS_air_outdoor', 'Noise_air_outdoor', 'PM10_air_classroom',
       'PM2.5_air_classroom', 'temp_air_classroom', 'RH_air_classroom',
       'AH_air_classroom', 'CO2_air_classroom', 'VOCS_air_classroom',
       'Noise_air_classroom', 'ΔT'],
      dtype='object')

In [ ]:
dataset = dataset[['tt', '0.3_outdoor', '0.5_outdoor', '1.0_outdoor', '3.0_outdoor',
       '5.0_outdoor', '10.0_outdoor', '0.3_classroom', '0.5_classroom',
       '1.0_classroom', '3.0_classroom', '5.0_classroom', '10.0_classroom',
       'CO2_outdoor','CO2_classroom']]

In [ ]:
dataset['tt'] = pd.to_datetime(dataset['tt'], format="%d-%m-%Y %H:%M:%S")

## NuN값 처리

In [ ]:
dataset.isnull().sum()

tt                0
0.3_outdoor       0
0.5_outdoor       0
1.0_outdoor       0
3.0_outdoor       0
5.0_outdoor       0
10.0_outdoor      0
0.3_classroom     0
0.5_classroom     0
1.0_classroom     0
3.0_classroom     0
5.0_classroom     0
10.0_classroom    0
CO2_outdoor       0
CO2_classroom     0
dtype: int64

In [ ]:
for i in reversed(range(0,len(dataset))): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i+1 < len(dataset)):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i+1,'CO2_classroom']

for i in range(0,len(dataset)): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i > 0):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i-1,'CO2_classroom']

## 미세먼지, 초미세먼지 단위 맞춤 (Scaling)

In [ ]:
dataset['0.3_outdoor_scaled'] = dataset['0.3_outdoor'] * 0.3 ** 3
dataset['0.5_outdoor_scaled'] = dataset['0.5_outdoor'] * 0.5 ** 3
dataset['1.0_outdoor_scaled'] = dataset['1.0_outdoor'] * 1.0 ** 3
dataset['3.0_outdoor_scaled'] = dataset['3.0_outdoor'] * 3.0 ** 3
dataset['5.0_outdoor_scaled'] = dataset['5.0_outdoor'] * 5.0 ** 3
dataset['10.0_outdoor_scaled'] = dataset['10.0_outdoor'] * 10.0 ** 3

dataset['0.3_classroom_scaled'] = dataset['0.3_classroom'] * 0.3 ** 3
dataset['0.5_classroom_scaled'] = dataset['0.5_classroom'] * 0.5 ** 3
dataset['1.0_classroom_scaled'] = dataset['1.0_classroom'] * 1.0 ** 3
dataset['3.0_classroom_scaled'] = dataset['3.0_classroom'] * 3.0 ** 3
dataset['5.0_classroom_scaled'] = dataset['5.0_classroom'] * 5.0 ** 3
dataset['10.0_classroom_scaled'] = dataset['10.0_classroom'] * 10.0 ** 3

## 미세먼지/초미세먼지(내부,외부) 데이터 통합

In [ ]:
dataset['PM2.5_outdoor'] = np.sum(dataset.loc[:,'0.3_outdoor_scaled':'1.0_outdoor_scaled'], axis=1)
dataset['PM10_outdoor'] =  np.sum(dataset.loc[:,'3.0_outdoor_scaled':'10.0_outdoor_scaled'], axis=1)

dataset['PM2.5_classroom'] = np.sum( dataset.loc[:,'0.3_classroom_scaled':'1.0_classroom_scaled'], axis=1)
dataset['PM10_classroom'] = np.sum( dataset.loc[:,'3.0_classroom_scaled':'10.0_classroom_scaled'], axis=1)

In [ ]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

In [ ]:
# PM_fig = go.Figure()
# from plotly.subplots import make_subplots
# PM_fig = make_subplots(specs=[[{"secondary_y": True}]])

# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_outdoor'],line=dict(color='red'), mode='lines',name='PM2.5_outdoors'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_outdoor'],line=dict(color='blue'),mode='lines',name='PM10_outdoors'),secondary_y=True,)
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_classroom'],line=dict(color='orange'),mode='lines',name='PM2.5_classroom'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_classroom'],line=dict(color='skyblue'),mode='lines',name='PM10_classroom'),secondary_y=True,)

# PM_fig.update_yaxes(title_text="PM 2.5", secondary_y=False)
# PM_fig.update_yaxes(title_text="PM10", secondary_y=True)

## 교실 내 변화량 column생성

In [ ]:
for i in range(1,len(dataset)) :
    dataset.loc[i,'PM2.5_classroom_change'] = dataset.loc[i,'PM2.5_classroom'] - dataset.loc[i-1,'PM2.5_classroom']
    dataset.loc[i,'PM10_classroom_change'] = dataset.loc[i,'PM10_classroom'] - dataset.loc[i-1,'PM10_classroom']
    dataset.loc[i,'CO2_classroom_change'] = dataset.loc[i,'CO2_classroom'] - dataset.loc[i-1,'CO2_classroom']

dataset.loc[0,'PM2.5_classroom_change'] =dataset.loc[1,'PM2.5_classroom_change']
dataset.loc[0,'PM10_classroom_change']  =dataset.loc[1,'PM10_classroom_change'] 
dataset.loc[0,'CO2_classroom_change']   =dataset.loc[1,'CO2_classroom_change']  

In [ ]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

# 스케일링 (일반화)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_data = scaler.fit_transform(dataset[['CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change']])
scaled_data = pd.DataFrame(data=scaled_data,columns=['CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change'])
scaled_data.insert(loc=0,column='tt',value=dataset['tt'])
scaled_data

,tt,CO2_classroom_change,PM2.5_classroom_change,PM10_classroom_change
0,2019-12-09 17:21:58,-0.014231,-0.017581,-0.794181
1,2019-12-09 17:22:58,-0.014231,-0.017581,-0.794181
2,2019-12-09 17:23:58,-0.014231,-0.094764,-0.048080
3,2019-12-09 17:24:58,-0.028180,-0.099106,-0.434727
4,2019-12-09 17:25:58,-0.014231,-0.489424,1.557173
...,...,...,...,...
4093,2019-12-12 13:34:59,-0.042129,-0.134481,0.009822
4094,2019-12-12 13:35:59,-0.097924,-0.334531,-0.092669
4095,2019-12-12 13:36:59,-0.125822,-0.371687,-0.577431
4096,2019-12-12 13:37:59,-0.097924,-0.162493,0.060514


In [ ]:
CO2_mean = np.mean(pd.concat([dataset['CO2_outdoor'],dataset['CO2_classroom']]))
CO2_std = np.std(pd.concat([dataset['CO2_outdoor'],dataset['CO2_classroom']]))
PM25_mean = np.mean(pd.concat([dataset['PM2.5_outdoor'],dataset['PM2.5_classroom']]))
PM25_std = np.std(pd.concat([dataset['PM2.5_outdoor'],dataset['PM2.5_classroom']]))
PM10_mean = np.mean(pd.concat([dataset['PM10_outdoor'],dataset['PM10_classroom']]))
PM10_std = np.std(pd.concat([dataset['PM10_outdoor'],dataset['PM10_classroom']]))
print(CO2_mean,CO2_std,PM25_mean,PM25_std,PM10_mean,PM10_std, sep ='\n')

622.9213030746706
372.5139474888872
15877648.532468928
16523436.237171357
19149736.411298193
31320775.293031055


In [ ]:
scaled_data['CO2_outdoor']    = (dataset['CO2_outdoor'] - CO2_mean) / CO2_std
scaled_data['PM2.5_outdoor']  = (dataset['PM2.5_outdoor'] - PM25_mean) / PM25_std
scaled_data['PM10_outdoor']   = (dataset['PM10_outdoor'] - PM10_mean) / PM10_std
scaled_data['CO2_classroom']  = (dataset['CO2_classroom'] - CO2_mean) / CO2_std
scaled_data['PM2.5_classroom']= (dataset['PM2.5_classroom'] - PM25_mean) / PM25_std
scaled_data['PM10_classroom'] = (dataset['PM10_classroom'] - PM10_mean) / PM10_std

scaled_data = scaled_data[['tt', 
 'CO2_outdoor',         'PM2.5_outdoor',         'PM10_outdoor',
 'CO2_classroom',       'PM2.5_classroom',       'PM10_classroom',
 'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change'
 ]]

In [ ]:
scaled_data

,tt,CO2_outdoor,PM2.5_outdoor,PM10_outdoor,CO2_classroom,PM2.5_classroom,PM10_classroom,CO2_classroom_change,PM2.5_classroom_change,PM10_classroom_change
0,2019-12-09 17:21:58,0.134434,0.359729,0.047473,-0.136696,-0.208164,0.209294,-0.014231,-0.017581,-0.794181
1,2019-12-09 17:22:58,0.118328,0.391758,0.017266,-0.139381,-0.208881,0.029177,-0.014231,-0.017581,-0.794181
2,2019-12-09 17:23:58,0.070007,0.362216,0.055783,-0.142065,-0.212139,0.018223,-0.014231,-0.094764,-0.048080
3,2019-12-09 17:24:58,-0.136696,0.355291,0.130061,-0.147434,-0.215540,-0.080395,-0.028180,-0.099106,-0.434727
4,2019-12-09 17:25:58,-0.227968,0.344928,0.136913,-0.150119,-0.231793,0.272608,-0.014231,-0.489424,1.557173
...,...,...,...,...,...,...,...,...,...,...
4093,2019-12-12 13:34:59,-0.437356,-0.697259,-0.322916,0.174701,-0.740177,0.169480,-0.042129,-0.134481,0.009822
4094,2019-12-12 13:35:59,-0.429303,-0.702589,-0.356757,0.155910,-0.751330,0.148416,-0.097924,-0.334531,-0.092669
4095,2019-12-12 13:36:59,-0.434672,-0.701170,-0.317906,0.131750,-0.763706,0.017443,-0.125822,-0.371687,-0.577431
4096,2019-12-12 13:37:59,-0.472254,-0.717380,-0.395645,0.112959,-0.769194,0.031110,-0.097924,-0.162493,0.060514


In [ ]:
scaled_data.isnull().sum()

tt                        0
CO2_outdoor               0
PM2.5_outdoor             0
PM10_outdoor              0
CO2_classroom             0
PM2.5_classroom           0
PM10_classroom            0
CO2_classroom_change      0
PM2.5_classroom_change    0
PM10_classroom_change     0
dtype: int64

In [ ]:
scaled_data.to_csv(choosefile+'_for_predict.csv',index=False)